In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np 
import pandas as pd
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


학습 데이터를 만든다.  

In [2]:
# xData => [공부 시간, 과외 시간]
temp = [[2, 3], [4, 3], [6, 4], [8, 6], [10, 7], [12, 8], [14, 9]]
print(temp)
print(type(temp))

xData = np.array(temp)
print(xData)
print(type(xData))

[[2, 3], [4, 3], [6, 4], [8, 6], [10, 7], [12, 8], [14, 9]]
<class 'list'>
[[ 2  3]
 [ 4  3]
 [ 6  4]
 [ 8  6]
 [10  7]
 [12  8]
 [14  9]]
<class 'numpy.ndarray'>


In [3]:
# yData => [합격 여부] => 실제값
temp = [0, 0, 0, 1, 1, 1, 1]
print(temp)
print(type(temp))

yData = np.array(temp)
print(yData)
# reshape() : 넘파이에서 데이터는 그대로 유지한 채 배열의 형태(차원)을 변경한다. 1행 7열 넘파이 배열 => 7행 1열 넘파이 배열
yData = np.array(temp).reshape(7, 1)
print(yData)
print(type(yData))

[0, 0, 0, 1, 1, 1, 1]
<class 'list'>
[0 0 0 1 1 1 1]
[[0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]]
<class 'numpy.ndarray'>


xData와 yData를 저장할 placeholder를 만든다.

In [4]:
# placeholder에 넘파이 배열을 대입하는 경우 shape 속성을 이용해 대입되는 넘파이 배열의 차원을 지정해줘야 한다.
# [None, 2] => placeholder에 대입되는 넘파이 배열의 행의 개수는 몇개라도 상관이 없고 열의 개수는 무조건 2개이다.
X = tf.placeholder(dtype=tf.float32, shape=[None, 2]) # xData가 대입될 placeholder
Y = tf.placeholder(dtype=tf.float32, shape=[None, 1]) # yData가 대입될 placeholder

기울기(가중치) a와 y절편(바이어스) b를 임의의 값으로 정한다.

In [5]:
a = tf.Variable(tf.random_uniform([2, 1]), dtype=tf.float32) # 난수를 2행 1열로 발생시킨다.
b = tf.Variable(tf.random_uniform([1]), dtype=tf.float32)
sess = tf.Session()
sess.run(tf.global_variables_initializer())
print('a1: {}, a2: {}, b: {}'.format(sess.run(a)[0], sess.run(a)[1], sess.run(b)))
print('{}'.format(sess.run(a)))
print('{}'.format(sess.run(b)))

a1: [0.9426694], a2: [0.6399087], b: [0.6677877]
[[0.9426694]
 [0.6399087]]
[0.6677877]


시그모이드 방정식, 오차 함수, 경하 하강

In [6]:
# matmul() : tensorflow에서 행렬의 곱셈을 계산하는 메소드
# sigmoid() : tensorflow에서 시그모이드 방정식을 계산하는 메소드
y = tf.sigmoid(tf.matmul(X, a) + b) # 시그모이드 방정식
loss = -tf.reduce_mean(Y * tf.log(y) + (1 - Y) * tf.log(1 - y)) # 오차 함수
gradient_descent = tf.train.GradientDescentOptimizer(0.1).minimize(loss) # 경사 하강

sigmoid() 메소드의 실행 결과(y)가 0.5 이상이면 1을 0.5 미만이면 0을 리턴한다.

In [7]:
sess = tf.Session()
# cast(캐스팅 할 데이터, dtype=캐스팅 될 데이터 타입) : tensorflow에서 형변환을 실행하는 메소드
# predicted = tf.cast(tf.constant([1.9, 2.1]), dtype=tf.int32) # 실수를 정수 데이터로 변환한다.
# print(sess.run(predicted)) # [1 2]
# cast() 메소드의 캐스팅할 데이터가 boolean 타입이면 True는 1로 False는 0으로 캐스팅한다.
predicted = tf.cast(0.4 >= 0.5, dtype=tf.int32)
print(sess.run(predicted)) # 0
predicted = tf.cast(0.5 >= 0.5, dtype=tf.int32)
print(sess.run(predicted)) # 1

0
1


sigmoid() 메소드를 실행한 예상값을 계산한다.

In [8]:
predicted = tf.cast(y >= 0.5, dtype=tf.float32) # 예측값, sigmoid() 메소드를 실행한 결과(y)를 0 또는 1로 변환한다.
# 예측값과 실제값이 일치하는 정도(정확도)를 계산한다.
# equal() : tensorflow에서 인수로 지정된 값이 같으면 True, 다르면 False를 리턴하는 메소드
# equal() 메소드로 예측값(predicted)과 실제값(Y)이 같은가 비교 후 True 또는 False 결과를 1 또는 0으로 캐스팅하고 시행된 전체
# 평균을 계산한다.
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.int32))

학습을 시킨다.

In [20]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(3001):
    # sess.run([기울기, y절편, 오차함수, 경사하강], feed_dict={X: 입력 데이터, Y: 입력 데이터에 따른 실제값})
    a_, b_, loss_, _ = sess.run([a, b, loss, gradient_descent], feed_dict={X: xData, Y: yData})
    if i % 300 == 0:
        # Epoch: 학습 횟수, a1: 공부 시간 기울기, a2: 과외 시간 기울기, b: y절편, loss: 오차 함수
        print('Epoch: %4d, a1: %9.5f, a2: %9.5f, b: %9.5f, loss: %9.5f' % (i, a_[0], a_[1], b_, loss_))
        
# 테스트 데이터를 만든다.
# new_x = np.array([6, 5]) # 테스트 데이터가 1차원 넘파이 배열이기 때문에 행렬의 곱셈 연산이 실행되지 않는다.
# print(new_x) # [6 5] => 1차원
# reshape() 메소드를 이용해서 1행 2열인 2차원 넘파이 배열로 테스트 데이터를 만들어야 한다.
new_x = np.array([6, 5]).reshape(1, 2)
# print(new_x) # [[6, 5]] => 2차원

# sigmoid() 메소드의 연산 결과에 테스트 데이터를 대입해서 연산한다.
result, new_y = sess.run([predicted, y], feed_dict={X: new_x})
print('공부 시간: %d, 과외 시간: %d' % (new_x[:, 0], new_x[:, 1]))
print('합격 여부: %d, 합격 확률: %f' % (result, new_y))
print('=' * 80)

for i in range(11):
    new_x = np.array([0, i]).reshape(1, 2)
    result, new_y = sess.run([predicted, y], feed_dict={X: new_x})
    print('공부 시간: %d, 과외 시간: %2d' % (new_x[:, 0], new_x[:, 1]), end=' ')
    print('합격 여부: %s, 합격 확률: %f%%' % ('  합격' if result == 1 else '불합격', new_y * 100))

Epoch:    0, a1:  -0.00256, a2:   0.01485, b:   0.75779, loss:   0.72302
Epoch:  300, a1:   0.86901, a2:  -0.66944, b:  -2.21059, loss:   0.28078
Epoch:  600, a1:   0.86341, a2:  -0.38118, b:  -3.73243, loss:   0.19945
Epoch:  900, a1:   0.76739, a2:  -0.03950, b:  -4.82790, loss:   0.15495
Epoch: 1200, a1:   0.65838, a2:   0.27841, b:  -5.69102, loss:   0.12623
Epoch: 1500, a1:   0.55501, a2:   0.56076, b:  -6.40456, loss:   0.10618
Epoch: 1800, a1:   0.46192, a2:   0.80901, b:  -7.01314, loss:   0.09146
Epoch: 2100, a1:   0.37953, a2:   1.02756, b:  -7.54386, loss:   0.08023
Epoch: 2400, a1:   0.30689, a2:   1.22099, b:  -8.01451, loss:   0.07141
Epoch: 2700, a1:   0.24275, a2:   1.39340, b:  -8.43735, loss:   0.06431
Epoch: 3000, a1:   0.18584, a2:   1.54818, b:  -8.82127, loss:   0.05848
공부 시간: 6, 과외 시간: 5
합격 여부: 1, 합격 확률: 0.508665
공부 시간: 0, 과외 시간:  0 합격 여부: 불합격, 합격 확률: 0.014758%
공부 시간: 0, 과외 시간:  1 합격 여부: 불합격, 합격 확률: 0.069350%
공부 시간: 0, 과외 시간:  2 합격 여부: 불합격, 합격 확률: 0.325301%
공부 시간